In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification, make_regression

np.random.seed(42)
tf.random.set_seed(42)

def feedforward_example():
    print("\n--- Feedforward Neural Network Example ---")
    X, y = make_classification(
        n_samples=1000, 
        n_features=20, 
        n_informative=10, 
        n_classes=2, 
        random_state=42
    )
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy', 
        metrics=['accuracy']
    )
    print("Training Feedforward Neural Network...")
    history = model.fit(
        X_train_scaled, y_train, 
        validation_split=0.2, 
        epochs=10, 
        batch_size=32,
        verbose=1 
    )

    test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)
    print(f"\nFeedforward Network - Test Accuracy: {test_accuracy:.4f}")
    
    return model

def cnn_mnist_example():
    print("\n--- Convolutional Neural Network (MNIST) Example ---")

    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

    X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255
    X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255
    
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print("Training Convolutional Neural Network...")
    history = model.fit(
        X_train[:5000], y_train[:5000], 
        validation_split=0.2, 
        epochs=5,
        batch_size=32,
        verbose=1  
    )
    test_loss, test_accuracy = model.evaluate(X_test[:1000], y_test[:1000], verbose=1)
    print(f"\nCNN - Test Accuracy: {test_accuracy:.4f}")
    return model

def rnn_time_series_example():
    print("\n--- Recurrent Neural Network (Time Series) Example ---")

    def create_time_series_dataset(n_samples=1000, time_steps=50):
        time = np.linspace(0, 10, n_samples)
        series = np.sin(time) + np.random.normal(0, 0.1, n_samples)

        X, y = [], []
        for i in range(len(series) - time_steps):
            X.append(series[i:i+time_steps])
            y.append(series[i+time_steps])
        
        return np.array(X).reshape(-1, time_steps, 1), np.array(y)

    X, y = create_time_series_dataset()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = keras.Sequential([
        layers.LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
        layers.Dense(25, activation='relu'),
        layers.Dense(1)
    ])
    
    model.compile(
        optimizer='adam', 
        loss='mse',
        metrics=['mae']
    )

    print("Training Recurrent Neural Network...")
    history = model.fit(
        X_train, y_train, 
        validation_split=0.2,
        epochs=10, 
        batch_size=32,
        verbose=1  
    )
    
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
    print(f"\nRNN - Test Mean Absolute Error: {test_mae:.4f}")
    
    return model

def run_neural_network_examples():
    print("Running Neural Network Examples...")
    feedforward_model = feedforward_example()
    cnn_model = cnn_mnist_example()
    rnn_model = rnn_time_series_example()

run_neural_network_examples()
